In [1]:
import pandas as pd

In [2]:
train_df = pd.read_csv('hate_speech_train_khaiii.csv', index_col=[0])

In [68]:
train_df.head(10)

,comments,comments_pos,bias,contain_gender_bias,hate,pos_list
0,현재 호텔 주인 심정 아18 난 마른 하늘에 날 벼락 맞고 호텔 망하게 생겼는데...,현재/MAG 호텔/NNG 주인/NNG 심정/NNG 아/NNG 18/SN 나/VA ㄴ...,others,False,hate,"[현재/MAG, 호텔/NNG, 주인/NNG, 심정/NNG, 아/NNG, 18/SN,..."
1,한국적인 미인의 대표적인 분 너무나 곱고 아름다운 모습 그 모습 뒤...,한국/NNP 적/XSN 이/VCP ㄴ/ETM 미인/NNG 의/JKG 대표/NNG 적...,none,False,none,"[한국/NNP, 적/XSN, 이/VCP, ㄴ/ETM, 미인/NNG, 의/JKG, 대..."
2,못된 넘들 남의 고통을 즐겼던 넘들 이젠 마땅한 처벌을 받아야지 ...,못/MAG 되/VA ㄴ/ETM 넘/NNG 들/XSN 남/NNG 의/JKG 고통/NN...,none,False,hate,"[못/MAG, 되/VA, ㄴ/ETM, 넘/NNG, 들/XSN, 남/NNG, 의/JK..."
3,1 2화 어설 펐는데 3 4화 지나서부터는 갈수록 너무 재밌던 데,1/SN 2/SN 화/NNB 어설/NNG 푸/VV 었/EP 는데/EC 3/SN 4/...,none,False,none,"[1/SN, 2/SN, 화/NNB, 어설/NNG, 푸/VV, 었/EP, 는데/EC,..."
4,1 사람 얼굴 손톱으로 긁은 것은 인격 살해이고 2 동영상이 몰카냐 메걸리안 ...,1/SN 사람/NNG 얼굴/NNG 손톱/NNG 으로/JKB 긁/VV 은/ETM 것/...,gender,True,hate,"[1/SN, 사람/NNG, 얼굴/NNG, 손톱/NNG, 으로/JKB, 긁/VV, 은..."
5,10 8 진짜 이 승기랑 비교된다,10/SN 8/SN 진짜/NNG 이/MM 승기/NNG 랑/JKB 비교/NNG 되/X...,none,False,none,"[10/SN, 8/SN, 진짜/NNG, 이/MM, 승기/NNG, 랑/JKB, 비교/..."
6,100년 안에 남녀 간 성 전쟁 한번 크게 치룬 후 일부다처제 여성의 정치 참여 ...,100/SN 년/NNB 안/NNG 에/JKB 남녀/NNG 간/NNB 성/NNG 전쟁...,gender,True,hate,"[100/SN, 년/NNB, 안/NNG, 에/JKB, 남녀/NNG, 간/NNB, 성..."
7,10년 뒤 윤서인은 분명히 재평가 될 것임 말 하나 하나가 틀린 게 없음,10/SN 년/NNB 뒤/NNG 윤서인/NNP 은/JX 분명히/MAG 재평가/NNG...,none,False,none,"[10/SN, 년/NNB, 뒤/NNG, 윤서인/NNP, 은/JX, 분명히/MAG, ..."
8,10년 만에 재미를 느끼는 프로였는데 왜 니들 때문에 폐지를 해야 되냐,10/SN 년/NNB 만/NNB 에/JKB 재미/NNG 를/JKO 느끼/VV 는/E...,none,False,offensive,"[10/SN, 년/NNB, 만/NNB, 에/JKB, 재미/NNG, 를/JKO, 느끼..."
9,10년차 방탄팬인데 우리 방탄처럼 성공은 못하겠지만 일단 방탄의 부하가 되고 싶다는...,10/SN 년/NNB 차/NNB 방탄/NNG 팬/NNG 이/VCP ㄴ데/EC 우리/...,none,False,offensive,"[10/SN, 년/NNB, 차/NNB, 방탄/NNG, 팬/NNG, 이/VCP, ㄴ데..."


In [18]:
#stop_words = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다']

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(train_df.comments_pos) # khaiii로 형태소 분석한 코멘트를 tf-idf 수행
print(tfidf_matrix.shape)

(7893, 12197)


In [34]:
from sklearn.metrics.pairwise import linear_kernel


cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # 코사인 유사도

In [62]:
# 코멘트에 관한 인덱스 테이블 구축
indices = pd.Series(train_df.index, index=train_df.comments_pos).drop_duplicates()

In [48]:
def comments_cos(comments_pos, cosine_sim=cosine_sim):
    #입력한 코멘트로 부터 인덱스 가져오기
    idx = indices[comments_pos]

    # 모든 코멘트에 대해서 해당 코멘트와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 코멘트들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 코멘트를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 코멘트의 인덱스 받아옴
    comment_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_df = train_df.iloc[comment_indices].copy()
    result_df['score'] = [i[1] for i in sim_scores]
    
    # 필요없는 컬럼 삭제
    del result_df['comments_pos']
    del result_df['pos_list']

    # 가장 유사한 10개의 코멘트을 리턴
    return result_df

In [74]:
train_df[train_df['hate'] == 'hate'].tail()

,comments,comments_pos,bias,contain_gender_bias,hate,pos_list
7864,흑발 미녀 장난똥 때리냐 마녀란 말도 아깝다,흑발/NNG 미녀/NNG 장/VV 난똥/NNG 때리/VV 냐/EC 마녀/NNG 란/...,others,False,hate,"[흑발/NNG, 미녀/NNG, 장/VV, 난똥/NNG, 때리/VV, 냐/EC, 마녀..."
7879,히잌 기저귀 찬 년들이 군대 군대 주댕이 털고 자 빠졌노ㅋ,히/NNG 잌/SH 기저귀/NNG 차/VV ㄴ/ETM 년/NNB 들/XSN 이/JK...,gender,True,hate,"[히/NNG, 잌/SH, 기저귀/NNG, 차/VV, ㄴ/ETM, 년/NNB, 들/X..."
7880,히트곡이 없는데 추억팔이가 되는 놀라운 조선식 방송국 연예매니징ㅋㅋ 기획사 소속사 ...,히트곡/NNG 이/JKS 없/VA 는데/EC 추억팔이/NNG 가/JKC 되/VV 는...,others,False,hate,"[히트곡/NNG, 이/JKS, 없/VA, 는데/EC, 추억팔이/NNG, 가/JKC,..."
7881,히트작 이래 봐야 미사 발리 두 개 정도가 다고 연기력도 그저 그런 수준으로 배우...,히트작/NNG 이래/NNB 보/VV 아야/EC 미사/NNG 발리/NNP 두/MM 개...,none,False,hate,"[히트작/NNG, 이래/NNB, 보/VV, 아야/EC, 미사/NNG, 발리/NNP,..."
7887,힘내세요 한고은님 btw 아래 한 남충 글 뭐냐 풉 평생 연애 하나 못하고 ...,힘/NNG 내/VA 시/EP 어요/EC 한고은/NNP 님/XSN btw/SL 아래/...,gender,True,hate,"[힘/NNG, 내/VA, 시/EP, 어요/EC, 한고은/NNP, 님/XSN, btw..."


In [85]:
comments_cos(train_df['comments_pos'][7887])  # 힘내세요 한고은님 btw 아래 한남충 글 뭐냐 풉 평생 연애 하나 못하고 ... hate

,comments,bias,contain_gender_bias,hate,score
2886,발 빨구 싶다 이 뻐 핰,none,False,hate,0.496860
3311,산이 힘내자 맞는 말만 했다,none,False,none,0.468548
1309,꼴깝을 떨고 있네 ㅋ,none,False,hate,0.456055
4280,애들 코 묻은 돈 뺏어가네,none,False,none,0.446455
6977,진짜 멋있다,none,False,none,0.400861
2761,민상아 처 낟 잘 보내고 와,none,False,none,0.396558
5564,응 빅뱅 오빠들이 깔아놓은 길 가면서 대단 한 척 하지마,others,False,offensive,0.390787
7243,코 했네,none,False,offensive,0.388789
7190,축하드려 요 ㅎ,none,False,none,0.376715
2280,때려 죽여도 기러 기 짓은 하지 말아라 여편 내들 외국 나가면 100 코쟁이들과 ...,gender,True,hate,0.372034


In [83]:
comments_cos(train_df['comments_pos'][3])  # 1, 2화 어설 펐는데 3, 4화 지나서부터는 갈수록 너무 재밌던 데... none

,comments,bias,contain_gender_bias,hate,score
3560,솔까 말 1위 2위 이 해안됨 3위 4위가 더 잘했다,none,False,none,0.555057
27,1부는 눈 썩는 거 같아서 2부만 봤다 졸라 웃기더 만 ㅋㅋ,none,False,none,0.513087
2403,만 17살이면 고 3인데 2살 차이나면 만날 수도 잇지,none,False,none,0.481296
58,2박2일은 뭐니 기레기야,none,False,hate,0.466992
4898,연기 쩌는 중 2,none,False,none,0.445321
23,1년 연예 ㅋㅋ 2년 봅니다,none,False,offensive,0.428442
6786,좋다 X1,none,False,none,0.418015
4699,여자가 시집가면 귀머거리 3년 벙어리 3년 장님 3년이라는 말도 있듯이 참고 살다 ...,gender,True,offensive,0.417737
4474,어쩐지 1위부터 와 꾸가 빻 고 나머지들 와 꾸가 개빻았길래 이 게 아이돌 이야 싶...,others,False,hate,0.414492
970,근데 5살 도 유모차 씀 애가 15년 초 출생인 걸로 아는데 5살이면 유모차...,none,False,none,0.398673


In [64]:
comments_cos(train_df['comments_pos'][8])  # 10년 만에 재미를 느끼는 프로였는데 왜 니들 때문에 폐지를 해야 되냐   offensive

,comments,bias,contain_gender_bias,hate,score
25,1박2일 재미 없고 이제 부도덕한 것만 생각나요 폐지하세요 학부모 입장에서 폐지 ...,none,False,none,0.420824
7827,확실이 여 자들이 투표하는 걸 좋아하네 4년째 이런 프로를 빨아주는 거 보니 ㅋㅋ ...,gender,True,offensive,0.395924
7359,틀 딱 들 프로 안 봐,others,False,hate,0.393575
1511,남들 다 갔다 오는 거,none,False,offensive,0.391347
3791,시청률 10 프로도 안 되고 무도 있었으면 대상도 못 받을 프로그램 이 문제인 이 ...,others,False,hate,0.387748
2699,뭘 저리 절절히 써 그냥 우린 너를 믿었는데 넌 거짓말을 쳤어 그러니 이 제 니...,none,False,offensive,0.383848
5255,왜 이리 악플만 있죠 전개가 답답하기 보단 연출이 세련된 거 같고 10분이라 느...,none,False,none,0.381854
4040,아무리 관리를 해도 늙어가는 건 어쩔 수 없다 젊어지려고 용을 써봐라 나이를 거슬러...,others,False,hate,0.368608
6762,졸 재미 없겠다,none,False,none,0.362535
6336,재미 1도 없다,none,False,none,0.360782
